# **IgFold**: Fast, accurate antibody structure prediction

Official notebook for [IgFold](https://www.biorxiv.org/content/10.1101/2022.04.20.488972): Fast, accurate antibody structure prediction from deep learning on massive set of natural antibodies.  The code, data, and weights for this work are made available for non-commercial use. For commercial inquiries, please contact `jruffolo[at]jhu.edu`.

In [28]:
#@title Input antibody Fv sequences then press `Runtime` -> `Run all`

import os
import sys

python_version = f"{sys.version_info.major}.{sys.version_info.minor}"

name = "my_antibody" #@param {type:"string"}
pred_dir = name
os.makedirs(pred_dir, exist_ok=True)

#@markdown Enter antibody sequences for structure prediction. To predict a nanobody structure (or an individual heavy or light chain), simply provide one sequence.
heavy_sequence = "EVQLVQSGPEVKKPGTSVKVSCKASGFTFMSSAVQWVRQARGQRLEWIGWIVIGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAPYCSSISCNDGFDIWGQGTMVTVS" #@param {type:"string"}
light_sequence = "DVVMTQTPFSLPVSLGDQASISCRSSQSLVHSNGNTYLHWYLQKPGQSPKLLIYKVSNRFSGVPDRFSGSGSGTDFTLKISRVEAEDLGVYFCSQSTHVPYTFGGGTKLEIK" #@param {type:"string"}

sequences = {}
if len(heavy_sequence) > 0:
    sequences["H"] = heavy_sequence
if len(light_sequence) > 0:
    sequences["L"] = light_sequence

#@markdown Perform structural refinement with OpenMM
do_refine = True #@param {type:"boolean"}
#@markdown Renumber predicted antibody structure (Chothia) with AbNumber
do_renum = False #@param {type:"boolean"}
#@markdown Use only a single model for predictions (instead of model ensemble)
single_model = False #@param {type:"boolean"}

In [20]:
#@title Install dependencies

PYTHON_VERSION = python_version

if not os.path.isfile("CONDA_READY"):
  print("installing conda...")
  os.system("wget -qnc https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh")
  os.system("bash Mambaforge-colab-Linux-x86_64.sh -bfp /usr/local")
  os.system("mamba config --set auto_update_conda false")
  os.system("touch CONDA_READY")

if not os.path.isfile("CODE_READY"):
  print("installing igfold...")
  torch_string = "torch==1.11.0+cu113 torchvision==0.12.0+cu113 -f https://download.pytorch.org/whl/torch_stable.html"
  os.system(f"pip3 install {torch_string}")
  os.system(f"pip install 'igfold>=0.3.0' {torch_string}")
  os.system("pip install -q --no-warn-conflicts 'py3Dmol>=2.0.1' matplotlib seaborn")
  os.system("touch CODE_READY")

if do_refine and not os.path.isfile("AMBER_READY"):
  print("installing amber...")
  os.system(f"mamba install -y -q -c conda-forge openmm=7.7.0 python='{PYTHON_VERSION}' pdbfixer 2>&1 1>/dev/null")
  os.system("touch AMBER_READY")

if do_renum and not os.path.isfile("ABNUMBER_READY"):
  print("installing abnumber...")
  os.system(f"mamba install -y -q -c bioconda abnumber python='{PYTHON_VERSION}' 2>&1 1>/dev/null")
  os.system("pip install pandas --force-reinstall")
  os.system("touch ABNUMBER_READY")

In [26]:
!wget https://ghproxy.com/https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip -O igfold.zip
!unzip igfold.zip
!cd IgFold-main && pip install .

--2025-05-27 03:28:53--  https://ghproxy.com/https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip
Resolving ghproxy.com (ghproxy.com)... 144.24.81.189
Connecting to ghproxy.com (ghproxy.com)|144.24.81.189|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ghproxy.link [following]
--2025-05-27 03:28:54--  https://ghproxy.link/
Resolving ghproxy.link (ghproxy.link)... 140.238.17.201
Connecting to ghproxy.link (ghproxy.link)|140.238.17.201|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1739 (1.7K) [text/html]
Saving to: ‘igfold.zip’

igfold.zip          100%[===================>]   1.70K  --.-KB/s    in 0s      

2025-05-27 03:28:55 (353 MB/s) - ‘igfold.zip’ saved [1739/1739]

Archive:  igfold.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will b

In [27]:
!wget https://ghproxy.com/https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip -O igfold.zip
!unzip igfold.zip
!pip install ./IgFold-main

--2025-05-27 03:30:56--  https://ghproxy.com/https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip
Resolving ghproxy.com (ghproxy.com)... 93.46.8.90
Connecting to ghproxy.com (ghproxy.com)|93.46.8.90|:443... failed: Connection timed out.
Retrying.

--2025-05-27 03:33:06--  (try: 2)  https://ghproxy.com/https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip
Connecting to ghproxy.com (ghproxy.com)|93.46.8.90|:443... failed: Connection timed out.
Retrying.

--2025-05-27 03:35:18--  (try: 3)  https://ghproxy.com/https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip
Connecting to ghproxy.com (ghproxy.com)|93.46.8.90|:443... failed: Connection timed out.
Retrying.

--2025-05-27 03:37:30--  (try: 4)  https://ghproxy.com/https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip
Connecting to ghproxy.com (ghproxy.com)|93.46.8.90|:443... failed: Connection timed out.
Retrying.

--2025-05-27 03:39:46--  (try: 5)  https://ghproxy.com/https://github.com/Gr

In [31]:
pip install igfold

  Using cached igfold-0.4.0-py3-none-any.whl.metadata (8.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB

In [2]:
!pip install igfold

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [3]:
from igfold import IgFoldRunner

/usr/local/lib/python3.11/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [10]:
with open("example.fasta", "w") as f:
    f.write(">Test\n")
    f.write("EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLEWVAYISSGGGNTYYADSVKGRFTISRDNAKNTLYLQMNSLRAEDTAVYYCAR\n")

In [11]:
import torch
from igfold import IgFoldRunner
heavy_sequence = "QVQLVQSGAEVKKPGASVKVSCKASGYTFTNYGMNWVRQAPGQGLEWMGRINPSDGSTYYNQKFKGKATLTVDTSSSTAYMELSSLRSEDTAVYYCAR"
light_sequence = "DIVLTQSPASLAVSLGQRATISCRASESVDNYGNTYLEWYQQKPGQSPKLLIYDTNNLASGVPDRFSGSGSGTDFTLTISSLEPEDFAVYYCQQYGSSPPTFGGGTKLEIK"
sequence = heavy_sequence + light_sequence  igfold = IgFoldRunner()
igfold.to("cuda" if torch.cuda.is_available() else "cpu")
with open("test.fasta", "w") as f:
    f.write(">test\n")
    f.write(sequence + "\n")
igfold.fold("test.fasta", "test.pdb", do_refine=False, renumber=False)

SyntaxError: invalid syntax (<ipython-input-11-37c740d69d1f>, line 5)

In [12]:
sequence = heavy_sequence + light_sequence
igfold = IgFoldRunner()


    The code, data, and weights for this work are made available for non-commercial use 
    (including at commercial entities) under the terms of the JHU Academic Software License 
    Agreement. For commercial inquiries, please contact awichma2[at]jhu.edu.
    License: https://github.com/Graylab/IgFold/blob/main/LICENSE.md
    
Loading 4 IgFold models...
Using device: cpu
Loading /usr/local/lib/python3.11/dist-packages/igfold/trained_models/IgFold/igfold_1.ckpt...


UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL transformers.models.bert.configuration_bert.BertConfig was not an allowed global by default. Please use `torch.serialization.add_safe_globals([BertConfig])` or the `torch.serialization.safe_globals([BertConfig])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about ty

In [13]:
!pip install git+https://github.com/Graylab/IgFold.git biopython==1.81
!wget -O igfold_trained_weights.pt https://dl.fbaipublicfiles.com/IgFold/igfold_trained_weights.pt
heavy_sequence = "QVQLVQSGAEVKKPGASVKVSCKASGYTFTNYGMNWVRQAPGQGLEWMGRINPSDGSTYYNQKFKGKATLTVDTSSSTAYMELSSLRSEDTAVYYCAR"
light_sequence = "DIVLTQSPASLAVSLGQRATISCRASESVDNYGNTYLEWYQQKPGQSPKLLIYDTNNLASGVPDRFSGSGSGTDFTLTISSLEPEDFAVYYCQQYGSSPPTFGGGTKLEIK"
sequence = heavy_sequence + light_sequence
with open("test.fasta", "w") as f:
    f.write(">test\n")
    f.write(sequence + "\n")
import torch
from igfold import IgFoldRunner
igfold = IgFoldRunner(model_weights_path="igfold_trained_weights.pt")
igfold.to("cuda" if torch.cuda.is_available() else "cpu")
igfold.fold("test.fasta", "test.pdb", do_refine=False, renumber=False)

  Cloning https://github.com/Graylab/IgFold.git to /tmp/pip-req-build-cmqpcd23
  Running command git clone --filter=blob:none --quiet https://github.com/Graylab/IgFold.git /tmp/pip-req-build-cmqpcd23
  Resolved https://github.com/Graylab/IgFold.git to commit 85d9dcdbc7afb9c38c4427df656fc194a61bd6ee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.8 MB/s eta 0:00:00
  Attempting uninstall: biopython
    Found existing installation: biopython 1.85
    Uninstalling biopython-1.85:
      Successfully uninstalled biopython-1.85


--2025-05-27 10:20:44--  https://dl.fbaipublicfiles.com/IgFold/igfold_trained_weights.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.50, 13.35.7.128, 13.35.7.38, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.50|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2025-05-27 10:20:44 ERROR 403: Forbidden.



TypeError: IgFoldRunner.__init__() got an unexpected keyword argument 'model_weights_path'

In [6]:
from igfold import IgFoldRunner
heavy_sequence = "QVQLVQSGAEVKKPGASVKVSCKASGYTFTNYGMNWVRQAPGQGLEWMGRINPSDGSTYYNQKFKGKATLTVDTSSSTAYMELSSLRSEDTAVYYCAR"
light_sequence = "DIVLTQSPASLAVSLGQRATISCRASESVDNYGNTYLEWYQQKPGQSPKLLIYDTNNLASGVPDRFSGSGSGTDFTLTISSLEPEDFAVYYCQQYGSSPPTFGGGTKLEIK"
igfold = IgFoldRunner(device="cuda" if torch.cuda.is_available() else "cpu")
sequence = f">test_sequence\n{heavy_sequence}:{light_sequence}"
with open("test.fasta", "w") as f:
    f.write(sequence)

igfold.fold(
    fasta_path="test.fasta",
    output_path="test.pdb",
    do_refine=False,      renumber=False

SyntaxError: incomplete input (<ipython-input-6-3dd8d53af314>, line 12)

In [1]:
from igfold import IgFoldRunner

ModuleNotFoundError: No module named 'igfold'

In [32]:
python

NameError: name 'python' is not defined

In [ ]:
!wget https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip -O igfold.zip

--2025-05-27 04:37:12--  https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-05-27 04:37:12 ERROR 404: Not Found.



In [ ]:
!wget https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip -O igfold.zip

--2025-05-27 04:37:12--  https://github.com/Graylab-Lab/IgFold/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-05-27 04:37:12 ERROR 404: Not Found.



In [19]:
#@title Predict antibody structure with IgFold

if f"/usr/local/lib/python{python_version}/site-packages/" not in sys.path:
    sys.path.insert(0, f"/usr/local/lib/python{python_version}/site-packages/")

from igfold.utils.visualize import *
from igfold import IgFoldRunner

num_models = 1 if single_model else 4
igfold = IgFoldRunner(num_models=num_models)

pred_pdb = os.path.join(pred_dir, f"{name}.pdb")
pred = igfold.fold(
    pred_pdb,
    sequences=sequences,
    do_refine=do_refine,
    use_openmm=True,
    do_renum=do_renum,
)
show_pdb(pred_pdb, len(sequences), bb_sticks=False, sc_sticks=True, color="rainbow")

ModuleNotFoundError: No module named 'igfold'

In [17]:
#@title Plot per-residue predicted RMSD

prmsd_fig_file = os.path.join(pred_dir, f"{name}_prmsd.png")
plot_prmsd(sequences, pred.prmsd.cpu(), prmsd_fig_file, shade_cdr=do_renum, pdb_file=pred_pdb)

NameError: name 'plot_prmsd' is not defined

In [ ]:
#@title Show predicted structure with predicted RMSD

#@markdown Structure is colored from low (blue) to high (red) pRMSD.

show_pdb(pred_pdb, len(sequences), bb_sticks=False, sc_sticks=True, color="b")

In [ ]:
#@title Download results

#@markdown Download zip file containing structure prediction and annotation results. If download fails, results are also accessible from file explorer on the left panel of the notebook.

from google.colab import files
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!zip -FSr $name".result.zip" $pred_dir/ &> /dev/null
files.download(f"{name}.result.zip")